# Практическое задание: построение модели предсказания продаж в торговой сети

В этом практическом задании вам нужно будет построить модель, делающую долгосрочный прогноз продаж товаров в торговой сети.
Задание основано на контесте: https://www.kaggle.com/c/m5-forecasting-accuracy. 

В данном задании вы не только построите модель, но и спроектируете небольшую архитектуру, позволяющую эффективно пересчитывать предсказания модели на новых данных. 

**Внимание**. Любые скрипты, параметры запуска и т.п. можно редактировать в разумных пределах.

## Этап 1: Фильтрация дней 

Напомним, что в данной задаче вам нужно предсказать на 28 дней вперёд продажи 3049 товаров в 10 магазинах.

Скорее всего для обучения хорошей модели не потребуется информация за всю историю продаж продуктов, более того информация по достаточно давним продажам может даже вредить.
Для экономии памяти лучше всего  сразу избавиться от дней, которые не будут входить в обучение.
На этапе объединения всех табличных данных в одну, необходимо оставить только необходимое для обучения количество дней. 

Для того, чтобы сгенерировать такой массив, вам достаточно запустить скрипт `generate_melted_df`, аргумент `min_train_day` отвечает за первый день, который будет добавлен в выборку. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
! python feature_generation/generate_melted_df.py --min_train_day=1540

tcmalloc: large alloc 3314147328 bytes == 0x17e954000 @  0x7f8e74c33001 0x7f8e72585765 0x7f8e725e9bb0 0x7f8e725e9f37 0x7f8e72681f28 0x50a635 0x50cd96 0x509758 0x50a48d 0x50bfb4 0x509758 0x50a48d 0x50bfb4 0x509758 0x50a48d 0x50bfb4 0x509758 0x50a48d 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509042 0x594931 0x549e5f


## Этап 2: Генерация признаков (3 балла)

Для обучения модели понадобится сгенерировать признаки, содержащие информацию о прошлых продажах товара.
Пусть $y_i$ — количество продаж каждого отдельного ряда в момент времени i.
Для удобства изложения, введём для каждого признака обозначение через результат выполнения некоторой функции.

#### Признаки, которые необходимо добавить в модель

* Количество продаж $i$ дней назад (shift_i):

    $$
    {
    f(t, i, 1, 1) = y_{t-i} 
    }
    $$


* Среднее число продаж за $j$ дней на момент времени $i$ дней назад (mean_i_j): 
    
    $$
    {
    f(t, i, j, 1) = \frac{1}{j}\sum_{k=0}^{j-1} y_{t-i-k} 
    }
    $$

* Среднее число продаж за $j$ последних дней с периодом $d$ на момент времени $i$ дней назад (mean_i_j_period_d):
    
    $$ 
    {
    f(t, i, j, d) = \frac{1}{j}\sum_{k=0}^{j-1} y_{t-i-k * d} 
    }
    $$


* Экспоненциальное скользящее среднее (c коэффициентом $\beta$) числа продаж на момент времени $i$ дней назад (exp_mean_i_beta):

    $$
    {
    g(t, i, \beta, 1) = \frac{\sum\limits_{k=0}^{t - i} y_{t-i-k} \beta^k}{\sum\limits_{k=0}^{t - i} \beta^k}
    }
    $$
    
    
* Экспоненциальное скользящее среднее (c коэффициентом $\beta$) числа продаж с периодом $d$ на момент времени $i$ дней назад (exp_mean_i_beta_period_d):
    
    $$ 
    {
    g(t, i, \beta, d) = \frac{\sum\limits_{k=0}^{[(t-i)/d]} y_{t-i-k * d} \beta^{k}}{\sum\limits_{k=0}^{[(t-i)/d]} \beta^k}
    }
    $$

Обратите внимание, что некоторые признаки являются частными случаями других признаков. Однако, возможно, функцию для их вычисления следует написать отдельно, для оптимизации времени выполнения.

Реализацию генерации признаков необходимо выполнить в модуле `generate_past_demand_features`. После реализации признаки можно сгенерировать запустив скрипт из командной строки (посмотреть пример сопоставления параметров признакм можно внутри модуля).

#### Пример

In [ ]:
import json
import re


config = json.dumps({
    'shift_parameters': [7, 28],
    'mean_parameters': [(7, 7), (28, 7)],
    'mean_period_parameters': [(7, 7, 7), (28, 7, 7)],
    'exp_mean_parameters': [(7, 0.5)],
    'exp_mean_period_parameters': [(7, 0.5, 7)],
})

config = re.sub('"', '%', config)

In [ ]:
! python feature_generation/generate_past_demand_features.py -c="$config"

shift 7
shift 28
mean 7 7
mean 28 7
mean period 7 7 7
mean period 28 7 7
exp  7 0.5
exp period 7 0.5 7


## Этап 3: Разделение данных на обучение и контроль

Для обучения данных необходимо разбить данные на обучение, валидацию (для подбора гиперпараметров) и тест (для финальной оценки качества модели).
В качестве теста логично брать последние несколько дней, для валидации - несколько дней непосредственно перед тестом, и все остальное - для обучения.

Однако, данные отложенные для тестирования и валидации являются самыми ценными, т.к. идут непосредственно перед пресказываемыми данными. Поэтому, настроив гиперпараметры на валидации, необходимо расширить обучающие данные за счет валидации и обучить финальную модель с известными гиперпараметрами на расширенных данных. Это необходимо будет сделать перед засылкой результата модели в соревнование.

В модуле `convert_csv_to_numpy` реализовано два типа разбиения данных: train/validation/test и train/test для настройки гиперпараметров модели и для обучения финальной модели соответственно. Запустите модуль из командной строки, чтобы получить разбиения.

In [ ]:
! python feature_generation/convert_csv_to_numpy.py

tcmalloc: large alloc 2541993984 bytes == 0x26dc6000 @  0x7fd2ca2ce001 0x7fd2c7e34765 0x7fd2c7e97070 0x7fd2c7e98f0f 0x7fd2c7f2f508 0x50a635 0x50bfb4 0x509758 0x50a48d 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7fd2c9ec9b97 0x5b250a
tcmalloc: large alloc 2009276416 bytes == 0xbe602000 @  0x7fd2ca2cc1e7 0x7fd2c7e345e1 0x7fd2c7e97138 0x7fd2c7e97253 0x7fd2c7f22396 0x7fd2c7f227f8 0x50c29e 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7fd2c9ec9b97 0x5b250a
tcmalloc: large alloc 2364424192 bytes == 0xc8f5c000 @  0x7fd2ca2cc1e7 0x7fd2c7e345e1 0x7fd2c7e97138 0x7fd2c7e97253 0x7fd2c7f22396 0x7fd2c7f227f8 0x50c29e 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7fd2c9ec9b97 0x5b250a


## Этап 4: Пересчёт значений признаков (6 балла)

При вычислении предсказания на тестовые 28 дней вперёд возникает ситуация, когда значения некоторых признаков оказываются неизвестными. В этом случае, эти значения необходимо заполнять с помощью предсказаний модели, полученных для меньшего горизонта прогнозирования. Например, признак количество продаж один день назад, будет вычисляться как предсказание модели в предыдущий день.

В этой части для каждого из признаков необходимо реализовать функцию пересчёта значений признака с учётом поступивших новых предсказаний модели. Заметим, что некоторые признаки можно легко пересчитать только имея информацию о других признаках.

**Задача 1**. Дано значение признака $f(t,i,j, d)$ в моменты времени от $\tau$ до $\tau + k$. Какие признаки необходимо иметь в модели, чтобы за $O(1)$ можно было пересчитать $f(\tau+k+1,i,j,d)$? Приведите формулу пересчёта ниже.

**Задача 2**. Дано значение признака $g(t,i,\beta, d)$ в моменты времени от $\tau$ до $\tau + k$. Какие признаки необходимо иметь в модели, чтобы за $O(1)$ можно было пересчитать $g(\tau+k+1,i,\beta,d)$? Приведите формулу пересчёта ниже.

**Внимание**. Не забудьте учесть случаи разных $k$!

## Решение

Есть два случая - когда есть $f(t,i,j,d)$ в предыдущий период, т.е. $f(\tau+k+1 - d,i,j,d + i)$, т.е. $ k \geq d$, тогда достаточно знать $y_{\tau + k + 1 - i - (j + 1) * d}$ - первый член суммы $f(\tau+ k + 1 - d,i,j,d)$ и если в модели есть признак $f(t,m,1,1)$ , тогда достаточно чтобы $ k \geq (j + 1) * d + m + i$, и формула будет выглядеть так:
$$f(\tau+k+1,i,j,d) = (f(\tau+k+1 - d,i,j,d) * j - y_{\tau + k + 1 - (j + 1) * d} + y_{\tau + k + 1 - i}) / j$$

т.е. еще нужно знать $y_{\tau + k + 1 - i}$

если же $ k \leq d + i$ - тогда  таким способом быстро пересчитать не получится и нужно иметь явно еще и признак $f(\tau+k+1 - d,i,j,d)$

Аналогично предыдущему при $k \leq d + i$
нужно иметь $g(\tau + k + 1 - d, i, \beta, d)$, при $k \geq d$ у нас есть $ g(\tau + k + 1 - d, i, \beta, d)$ и формула такая:
$$ g(\tau + k + 1, i, \beta, d) =( g(\tau + k + 1 - d, i, \beta, d) * (\sum_{l = 0}^{[\frac{\tau + k + 1 -d - i}{d}]} \beta^l) * beta + y_{\tau + k + 1 - i}  ) / (\sum_{l = 0}^{[\frac{\tau + k + 1 - i}{d}]} \beta^l) $$ т.е. нужно знать $y_{\tau + k + 1 - i}$ и количество дней за которые считалось экспоненциальное среднее, поэтому в класс пересчета я добавил первый день истории для каждого товара.

Реализацию пересчёта признаков необходимо выполнить в модуле `feature_recalculation`. Категориальные признаки в этом модуле пересчитывать не нужно.

## Этап 5: Построение и валидация модели

В качестве основной метрики задачи испольузется функционал WRMSSE (подробнее о нём можно узнать из соответствующей презентации к семинару). Логично, при обучении модели настраиваться именно на конкретный функционал. 

Чтобы не тратить каждый раз память на подготовку данных для подсчёта функционала по метрике, выполните скрипт `utils/generate_evaluators` и сохраните данные для эвалюаторов на диск.

In [ ]:
! python utils/generate_evaluators.py

100% 42840/42840 [00:07<00:00, 5895.60it/s]
100% 42840/42840 [00:07<00:00, 5955.41it/s]


В качестве основной модели построения предсказаний вам предлагается использовать градиентный бустинг (рекомендуем использовать lightgbm для быстроты проведения экспериментов). 

Для сравнения результатов моделей вам необходимо использовать следующую схему:

1. Обучение бустинга на train выборке на функционал MSE, валидация на val выборке по функционалу WRMSE.
2. Получение предсказаний на test выборке, с использованием рекурсивного пересчёта признаков

Обратите внимание, что на шаге 1 использовать рекурсивный пересчёт не нужно, т.к. это сильно замедлит обучение (более того, для ускорения обучения рекомендуется измерять значения функционала на валидации лишь раз в $n$ итераций). Однако, на тестовых данных всё должно быть максимально приближенно к реальности, поэтому признаки необходимо пересчитывать на ходу.

**Совет**. Чтобы точно получить правильный результат на тесте, можно после считывания данных специально занулить "неизвестные" значения признаков.

## Этап 6: Эксперименты (5)

### 1 Эксперимент

Сравните по указанной выше схеме результаты две модели.

1. Модель с двумя вещественными признаками:

    $f(t, i, 1, 1), i \in \{7, 28\}$


2. Модель с восемью вещественными признаками:

    $ f(t, i, 1, 1)$, $i \in [1, 7] \cup \{28\}$

Сделайте выводы о качестве работы моделей на валидации и отложенной выборке и заполните таблицу ниже:

| WRMSSE        | val без пересчёта | test без пересчёта | test с пересчётом |
|---------------|-------------------|--------------------|-------------------|
| Первая модель |  0.956918               |0.851172                   |1.06971                   |
| Вторая модель | 0.8951                  |1.417138                    |1.41713                  |

Ниже пример запуска экспериментов:  

In [ ]:
## ваш код для подсчёта нужных признаков

import json
import re


config = json.dumps({
    'shift_parameters': [1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 35, 42, 7 * 7],
    'mean_parameters': [],
    'mean_period_parameters': [(7, 7, 7)],
    'exp_mean_parameters': [],
    'exp_mean_period_parameters': [(7, 0.5, 7)],
})

config = re.sub('"', '%', config)

In [ ]:
! python feature_generation/generate_past_demand_features.py -c="$config"

shift 1
shift 2
shift 3
shift 4
shift 5
tcmalloc: large alloc 1089634304 bytes == 0xe9dc0000 @  0x7fb57061f1e7 0x7fb56e1855e1 0x7fb56e1e9c78 0x7fb56e1e9d93 0x7fb56e287ea8 0x7fb56e288704 0x7fb56e288852 0x566d63 0x59fc4e 0x7fb56e1d54ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x507d64 0x588d41 0x59fc4e 0x7fb56e1d54ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x509758
shift 6
tcmalloc: large alloc 1271242752 bytes == 0x7d366000 @  0x7fb57061f1e7 0x7fb56e1855e1 0x7fb56e1e9c78 0x7fb56e1e9d93 0x7fb56e287ea8 0x7fb56e288704 0x7fb56e288852 0x566d63 0x59fc4e 0x7fb56e1d54ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x507d64 0x588d41 0x59fc4e 0x7fb56e1d54ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x509758
shift 7
tcmalloc: large alloc 1452843008 bytes == 0x114c1a000 @  0x7fb57061f1e7 0x7fb56e1855e1 0x7fb56e1e9c78 0x7fb56e1e9d93 0x7fb56e287ea8 0x7fb56e288704 0x7fb56e288852 0x56

In [ ]:
! python feature_generation/convert_csv_to_numpy.py

tcmalloc: large alloc 5992980480 bytes == 0x1f0ac000 @  0x7f42083fc001 0x7f4205f60765 0x7f4205fc4bb0 0x7f4205fc6a4f 0x7f420605d048 0x50a635 0x50bfb4 0x509758 0x50a48d 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f4207ff7b97 0x5b250a
tcmalloc: large alloc 5096153088 bytes == 0x1c3b24000 @  0x7f42083fa1e7 0x7f4205f605e1 0x7f4205fc4c78 0x7f4205fc4d93 0x7f420604fed6 0x7f4206050338 0x50c29e 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f4207ff7b97 0x5b250a
tcmalloc: large alloc 5546917888 bytes == 0x1c3b24000 @  0x7f42083fa1e7 0x7f4205f605e1 0x7f4205fc4c78 0x7f4205fc4d93 0x7f420604fed6 0x7f4206050338 0x50c29e 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f4207ff7b97 0x5b250a


In [ ]:
! python utils/generate_evaluators.py

100% 42840/42840 [00:07<00:00, 6044.05it/s]
100% 42840/42840 [00:07<00:00, 6050.50it/s]


In [ ]:
import numpy as np

In [ ]:
# здесь хранится соответствие колонок признакам
with open('data/final_feature_header.json') as f:
    header = json.load(f)

target_column = header.index('demand')
day_column = header.index('day_as_int')

X = {}
y = {}

for val_type in ['train', 'val', 'test']:
    X[val_type] = np.load(f'data/X_{val_type}_validation.npy')
    y[val_type] = np.copy(X[val_type][:, target_column])

In [ ]:
import pickle

In [ ]:
from utils.metric_calculation import WRMSSEEvaluator

In [ ]:
# загрузка вспомогательных сущностей

# соответствие категориальных фичей их кодам
with open('data/label_codes_dict.json', 'rb') as f:
    label_codes_dict = json.load(f)

# эвалюаторы
with open('evaluators/valid_evaluator.pkl', 'rb') as f:
    val_evaluator = pickle.load(f)
    
with open('evaluators/test_evaluator.pkl', 'rb') as f:
    test_evaluator = pickle.load(f)

Напишите класс для подачи в интерфейс LighGBM, позволяющий вычислять значение метрики, используя готовый evaluator. Не забудьте предусмотреть вычисление лишь раз в $n$ итераций.

In [ ]:
class WRMSSEEvalMetric:
    def __init__(self, evaluator, ids, verbose_step=5):
        self.evaluator = evaluator
        self.verbose_step = verbose_step
        self.current_step = 0
        self.current_answer = 0
        self.ids = ids
    
    def __call__(self, y_true, y_pred):
        if self.current_step == 0:
            self.current_answer = self.evaluator.score(y_pred.reshape(-1, 28)[ids])
            self.current_step += 1
        else:
            if self.current_step == self.verbose_step - 1:
                self.current_step = 0
            else:
                self.current_step +=1
        return 'WRMSSE', self.current_answer, False

In [ ]:
import pandas as pd
ids = pd.read_csv('data/sales_train_validation.csv')['id'].map(label_codes_dict['id']).values

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
regressor = LGBMRegressor(
    n_estimators=500,
    learning_rate=0.1,
    objective='mse',
    metric="None"
)
wrmsse_val_eval_metric = WRMSSEEvalMetric(
    val_evaluator,
    ids,
    verbose_step=10
)

In [ ]:
# Берем нужные признаки
features = ['shift_7', 'shift_28']
feature_index = []
for feature in features:
    feature_index.append(header.index(feature))

In [ ]:
regressor.fit(
   X['train'][:, feature_index], y['train'],
   eval_set=[(X['val'][:, feature_index], y['val'])],
   eval_metric=wrmsse_val_eval_metric,
   early_stopping_rounds=20,
   verbose=10,
)

Training until validation scores don't improve for 20 rounds.
[10]	valid_0's WRMSSE: 2.15321
[20]	valid_0's WRMSSE: 1.32993
[30]	valid_0's WRMSSE: 1.07926
[40]	valid_0's WRMSSE: 1.00029
[50]	valid_0's WRMSSE: 0.973825
[60]	valid_0's WRMSSE: 0.964485
[70]	valid_0's WRMSSE: 0.960798
[80]	valid_0's WRMSSE: 0.959521
[90]	valid_0's WRMSSE: 0.958747
[100]	valid_0's WRMSSE: 0.958302
[110]	valid_0's WRMSSE: 0.957989
[120]	valid_0's WRMSSE: 0.95779
[130]	valid_0's WRMSSE: 0.95769
[140]	valid_0's WRMSSE: 0.95757
[150]	valid_0's WRMSSE: 0.957541
[160]	valid_0's WRMSSE: 0.957411
[170]	valid_0's WRMSSE: 0.957364
[180]	valid_0's WRMSSE: 0.957281
[190]	valid_0's WRMSSE: 0.957071
[200]	valid_0's WRMSSE: 0.957018
[210]	valid_0's WRMSSE: 0.957003
[220]	valid_0's WRMSSE: 0.956918
[230]	valid_0's WRMSSE: 0.956931
Early stopping, best iteration is:
[211]	valid_0's WRMSSE: 0.956918


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              metric='None', min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=500, n_jobs=-1, num_leaves=31,
              objective='mse', random_state=None, reg_alpha=0.0, reg_lambda=0.0,
              silent=True, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0)

In [ ]:
from feature_generation.feature_recalculation import RecursivePredictor

In [ ]:
y_pred = regressor.predict(X['test'][:, feature_index])

In [ ]:
wrmsse_test_eval_metric = WRMSSEEvalMetric(
    test_evaluator,
    ids,
    verbose_step=10
)

In [ ]:
wrmsse_test_eval_metric(0, y_pred)

('WRMSSE', 0.8511726630590583, False)

In [ ]:
rp = RecursivePredictor(header, regressor, label_codes_dict, 1180, feature_index)

In [ ]:
y_pred = rp.get_recursive_prediction(X['test'])

In [ ]:
wrmsse_test_eval_metric = WRMSSEEvalMetric(
    test_evaluator,
    label_codes_dict,
    verbose_step=10
)

In [ ]:
wrmsse_test_eval_metric(0, y_pred)

('WRMSSE', 1.0697105858277571, False)

In [ ]:
# Берем нужные признаки
features = ['shift_1', 'shift_2', 'shift_3', 'shift_4', 'shift_5', 'shift_6', 'shift_7', 'shift_28']
feature_index = []
for feature in features:
    feature_index.append(header.index(feature))

In [ ]:
regressor = LGBMRegressor(
    n_estimators=500,
    learning_rate=0.1,
    objective='mse',
    metric="None"
)
wrmsse_val_eval_metric = WRMSSEEvalMetric(
    val_evaluator,
    ids,
    verbose_step=10
)
regressor.fit(
   X['train'][:, feature_index], y['train'],
   eval_set=[(X['val'][:, feature_index], y['val'])],
   eval_metric=wrmsse_val_eval_metric,
   early_stopping_rounds=10,
   verbose=10,
)

Training until validation scores don't improve for 10 rounds.
[2]	valid_0's WRMSSE: 2.15006
[4]	valid_0's WRMSSE: 2.15006
[6]	valid_0's WRMSSE: 2.15006
[8]	valid_0's WRMSSE: 2.15006
[10]	valid_0's WRMSSE: 2.15006
[12]	valid_0's WRMSSE: 1.29013
[14]	valid_0's WRMSSE: 1.29013
[16]	valid_0's WRMSSE: 1.29013
[18]	valid_0's WRMSSE: 1.29013
[20]	valid_0's WRMSSE: 1.29013
[22]	valid_0's WRMSSE: 1.02422
[24]	valid_0's WRMSSE: 1.02422
[26]	valid_0's WRMSSE: 1.02422
[28]	valid_0's WRMSSE: 1.02422
[30]	valid_0's WRMSSE: 1.02422
[32]	valid_0's WRMSSE: 0.943125
[34]	valid_0's WRMSSE: 0.943125
[36]	valid_0's WRMSSE: 0.943125
[38]	valid_0's WRMSSE: 0.943125
[40]	valid_0's WRMSSE: 0.943125
[42]	valid_0's WRMSSE: 0.916399
[44]	valid_0's WRMSSE: 0.916399
[46]	valid_0's WRMSSE: 0.916399
[48]	valid_0's WRMSSE: 0.916399
[50]	valid_0's WRMSSE: 0.916399
[52]	valid_0's WRMSSE: 0.907416
[54]	valid_0's WRMSSE: 0.907416
[56]	valid_0's WRMSSE: 0.907416
[58]	valid_0's WRMSSE: 0.907416
[60]	valid_0's WRMSSE: 0.9074

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              metric='None', min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=500, n_jobs=-1, num_leaves=31,
              objective='mse', random_state=None, reg_alpha=0.0, reg_lambda=0.0,
              silent=True, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0)

In [ ]:
y_pred = regressor.predict(X['test'][:, feature_index])
wrmsse_test_eval_metric = WRMSSEEvalMetric(
    test_evaluator,
    ids,
    verbose_step=10
)
wrmsse_test_eval_metric(0, y_pred)

('WRMSSE', 1.4171387573513932, False)

In [ ]:
rp = RecursivePredictor(header, regressor, label_codes_dict, 1180, feature_index)
y_pred = rp.get_recursive_prediction(X['test'])
wrmsse_test_eval_metric = WRMSSEEvalMetric(
    test_evaluator,
    ids,
    verbose_step=10
)
wrmsse_test_eval_metric(0, y_pred)

('WRMSSE', 1.4171387573513932, False)

### 2 Эксперимент

Сравните по указанной выше схеме результаты двух моделей:

1. Модель с признаками:

    $f(t, i, 1, 1)$, $i \in \{7, 14, 21, 28, 35, 42, 49\}$
    
    
2. Модель с признаками:

    $f(t, 7, 7, 7)$, $g(t, 7, 0.5, 7)$
    
Сделайте выводы о качестве работы моделей на валидации и отложенной выборке и заполните таблицу ниже:

| WRMSSE        | val без пересчёта | test без пересчёта | test с пересчётом |
|---------------|-------------------|--------------------|-------------------|
| Первая модель | 0.795869                  |  0.9408849                  | 0.9133               |
| Вторая модель |  0.79997                 |  1.202                  |1.2228                   |

In [ ]:
# Берем нужные признаки
features = ['shift_7', 'shift_14', 'shift_21', 'shift_28', 'shift_35', 'shift_42', 'shift_49']
feature_index = []
for feature in features:
    feature_index.append(header.index(feature))

In [ ]:
regressor = LGBMRegressor(
    n_estimators=500,
    learning_rate=0.1,
    objective='mse',
    metric="None"
)
wrmsse_val_eval_metric = WRMSSEEvalMetric(
    val_evaluator,
    ids,
    verbose_step=10
)
regressor.fit(
   X['train'][:, feature_index], y['train'],
   eval_set=[(X['val'][:, feature_index], y['val'])],
   eval_metric=wrmsse_val_eval_metric,
   early_stopping_rounds=20,
   verbose=10,
)

Training until validation scores don't improve for 20 rounds.
[10]	valid_0's WRMSSE: 2.14499
[20]	valid_0's WRMSSE: 1.23828
[30]	valid_0's WRMSSE: 0.948937
[40]	valid_0's WRMSSE: 0.85514
[50]	valid_0's WRMSSE: 0.822154
[60]	valid_0's WRMSSE: 0.809538
[70]	valid_0's WRMSSE: 0.805742
[80]	valid_0's WRMSSE: 0.802733
[90]	valid_0's WRMSSE: 0.801287
[100]	valid_0's WRMSSE: 0.80037
[110]	valid_0's WRMSSE: 0.799352
[120]	valid_0's WRMSSE: 0.799088
[130]	valid_0's WRMSSE: 0.798766
[140]	valid_0's WRMSSE: 0.798257
[150]	valid_0's WRMSSE: 0.798137
[160]	valid_0's WRMSSE: 0.797755
[170]	valid_0's WRMSSE: 0.797789
[180]	valid_0's WRMSSE: 0.797743
[190]	valid_0's WRMSSE: 0.797594
[200]	valid_0's WRMSSE: 0.797306
[210]	valid_0's WRMSSE: 0.797293
[220]	valid_0's WRMSSE: 0.797208
[230]	valid_0's WRMSSE: 0.79696
[240]	valid_0's WRMSSE: 0.7969
[250]	valid_0's WRMSSE: 0.796856
[260]	valid_0's WRMSSE: 0.796713
[270]	valid_0's WRMSSE: 0.796501
[280]	valid_0's WRMSSE: 0.796262
[290]	valid_0's WRMSSE: 0.7960

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              metric='None', min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=500, n_jobs=-1, num_leaves=31,
              objective='mse', random_state=None, reg_alpha=0.0, reg_lambda=0.0,
              silent=True, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0)

In [ ]:
y_pred = regressor.predict(X['test'][:, feature_index])
wrmsse_test_eval_metric = WRMSSEEvalMetric(
    test_evaluator,
    ids,
    verbose_step=10
)
wrmsse_test_eval_metric(0, y_pred)

('WRMSSE', 0.9408849642822643, False)

In [ ]:
rp = RecursivePredictor(header, regressor, label_codes_dict, 1180, feature_index)
y_pred = rp.get_recursive_prediction(X['test'])
wrmsse_test_eval_metric = WRMSSEEvalMetric(
    test_evaluator,
    ids,
    verbose_step=10
)
wrmsse_test_eval_metric(0, y_pred)

('WRMSSE', 0.9133989611191552, False)

In [ ]:
# Берем нужные признаки
features = ['mean_7_7_period_7', 'mean_7_0.500_period_7']
feature_index = []
for feature in features:
    feature_index.append(header.index(feature))

In [ ]:
regressor = LGBMRegressor(
    n_estimators=500,
    learning_rate=0.1,
    objective='mse',
    metric="None"
)
wrmsse_val_eval_metric = WRMSSEEvalMetric(
    val_evaluator,
    ids,
    verbose_step=10
)
regressor.fit(
   X['train'][:, feature_index], y['train'],
   eval_set=[(X['val'][:, feature_index], y['val'])],
   eval_metric=wrmsse_val_eval_metric,
   early_stopping_rounds=10,
   verbose=10,
)

Training until validation scores don't improve for 10 rounds.
[10]	valid_0's WRMSSE: 2.12933
[20]	valid_0's WRMSSE: 1.18961
[30]	valid_0's WRMSSE: 0.91975
[40]	valid_0's WRMSSE: 0.842103
[50]	valid_0's WRMSSE: 0.818209
[60]	valid_0's WRMSSE: 0.810097
[70]	valid_0's WRMSSE: 0.80685
[80]	valid_0's WRMSSE: 0.805288
[90]	valid_0's WRMSSE: 0.803528
[100]	valid_0's WRMSSE: 0.802434
[110]	valid_0's WRMSSE: 0.801941
[120]	valid_0's WRMSSE: 0.801644
[130]	valid_0's WRMSSE: 0.801403
[140]	valid_0's WRMSSE: 0.801291
[150]	valid_0's WRMSSE: 0.801217
[160]	valid_0's WRMSSE: 0.801105
[170]	valid_0's WRMSSE: 0.801061
[180]	valid_0's WRMSSE: 0.800818
[190]	valid_0's WRMSSE: 0.800752
[200]	valid_0's WRMSSE: 0.800604
[210]	valid_0's WRMSSE: 0.800572
[220]	valid_0's WRMSSE: 0.800482
[230]	valid_0's WRMSSE: 0.800292
[240]	valid_0's WRMSSE: 0.800205
[250]	valid_0's WRMSSE: 0.800185
[260]	valid_0's WRMSSE: 0.800162
[270]	valid_0's WRMSSE: 0.800112
[280]	valid_0's WRMSSE: 0.79997
Early stopping, best iterati

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              metric='None', min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=500, n_jobs=-1, num_leaves=31,
              objective='mse', random_state=None, reg_alpha=0.0, reg_lambda=0.0,
              silent=True, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0)

In [ ]:
y_pred = regressor.predict(X['test'][:, feature_index])
wrmsse_test_eval_metric = WRMSSEEvalMetric(
    test_evaluator,
    label_codes_dict,
    verbose_step=10
)
wrmsse_test_eval_metric(0, y_pred)

('WRMSSE', 1.202094902155587, False)

In [ ]:
rp = RecursivePredictor(header, regressor, label_codes_dict, 1180, feature_index)
y_pred = rp.get_recursive_prediction(X['test'])
wrmsse_test_eval_metric = WRMSSEEvalMetric(
    test_evaluator,
    label_codes_dict,
    verbose_step=10
)
wrmsse_test_eval_metric(0, y_pred)

('WRMSSE', 1.2228506684707083, False)

## Этап 7. Получение результата для засылки в контест (1 балл)

Пришло время заслать своё решение в соревнование.
Ваше решение должно преодолеть скор бейзлайна 0.565.

Преодолевшие скор бейзлайна могут получить бонусные баллы.
Ваш бонусный балл вычисляется по следующей функции:

`
int(np.linspace(5, 0, n_students)[your_place])
`

где `n_students` — количество студентов, преодолевших бейзлайн, `your_place` — ваше место среди студентов OzonMasters.

In [ ]:
## ваш код для подсчёта нужных признаков

import json
import re


config = json.dumps({
    'shift_parameters': [1, 7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98, 140, 280],
    'mean_parameters': [(1, 7), (8, 7), (14, 7), (28, 7), (14, 14), (14, 28)],
    'mean_period_parameters': [(7, 7, 7)],
    'exp_mean_parameters': [(1, 0.5), (1, 0.7), (1, 0.85), (14, 0.5), (14, 0.7), (14, 0.85)],
    'exp_mean_period_parameters': [(7, 0.5, 7)],
})

config = re.sub('"', '%', config)

In [ ]:
! python feature_generation/generate_past_demand_features.py -c="$config"

shift 1
shift 7
shift 14
shift 21
shift 28
shift 35
shift 42
shift 49
shift 56
shift 63
shift 70
shift 77
shift 84
shift 91
shift 98
shift 140
shift 280
mean 1 7
mean 8 7
mean 14 7
mean 28 7
mean 14 14
mean 14 28
mean period 7 7 7
tcmalloc: large alloc 2444222464 bytes == 0xa5244000 @  0x7f5b083611e7 0x7f5b05ec75e1 0x7f5b05f2bc78 0x7f5b05f2bd93 0x7f5b05fc9ea8 0x7f5b05fca704 0x7f5b05fca852 0x566d63 0x59fc4e 0x7f5b05f174ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x507d64 0x588d41 0x59fc4e 0x7f5b05f174ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x509758
tcmalloc: large alloc 2444222464 bytes == 0x136d42000 @  0x7f5b083611e7 0x7f5b05ec75e1 0x7f5b05f2bc78 0x7f5b05f2bd93 0x7f5b05fb6ed6 0x7f5b05fb7338 0x50c29e 0x507d64 0x509a90 0x50a48d 0x50cd96 0x509758 0x50a48d 0x50bfb4 0x509758 0x50a48d 0x50bfb4 0x509758 0x50a48d 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x509758 0x50a48d 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50bfb4
tcmalloc

In [ ]:
! python feature_generation/convert_csv_to_numpy.py

tcmalloc: large alloc 4790673408 bytes == 0x49d2a000 @  0x7f134f302001 0x7f134ce66765 0x7f134cecabb0 0x7f134cecca4f 0x7f134cf63048 0x50a635 0x50bfb4 0x509758 0x50a48d 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f134eefdb97 0x5b250a
tcmalloc: large alloc 3786702848 bytes == 0x1675e8000 @  0x7f134f3001e7 0x7f134ce665e1 0x7f134cecac78 0x7f134cecad93 0x7f134cf55ed6 0x7f134cf56338 0x50c29e 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f134eefdb97 0x5b250a
tcmalloc: large alloc 4456022016 bytes == 0x1675e8000 @  0x7f134f3001e7 0x7f134ce665e1 0x7f134cecac78 0x7f134cecad93 0x7f134cf55ed6 0x7f134cf56338 0x50c29e 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f134eefdb97 0x5b250a


In [ ]:
import numpy as np

In [ ]:
import pickle

In [ ]:
from utils.metric_calculation import WRMSSEEvaluator

In [ ]:
# здесь хранится соответствие колонок признакам
with open('data/final_feature_header.json') as f:
    header = json.load(f)

target_column = header.index('demand')
day_column = header.index('day_as_int')

X = {}
y = {}

for val_type in ['train', 'val', 'test']:
    X[val_type] = np.load(f'data/X_{val_type}_validation.npy')
    y[val_type] = np.copy(X[val_type][:, target_column])

In [ ]:
# загрузка вспомогательных сущностей

# соответствие категориальных фичей их кодам
with open('data/label_codes_dict.json', 'rb') as f:
    label_codes_dict = json.load(f)

# эвалюаторы
with open('evaluators/valid_evaluator.pkl', 'rb') as f:
    val_evaluator = pickle.load(f)
    
with open('evaluators/test_evaluator.pkl', 'rb') as f:
    test_evaluator = pickle.load(f)

In [ ]:
feature_index = list(range(5)) + list(range(7, 18)) + list(range(20, len(header)))

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
regressor = LGBMRegressor(
    n_estimators=500,
    learning_rate=0.1,
    num_leaves=19,
    objective='mse',
    metric="None"
)
wrmsse_val_eval_metric = WRMSSEEvalMetric(
    val_evaluator,
    ids,
    verbose_step=10
)
regressor.fit(
   X['train'][:, feature_index], y['train'],
   eval_set=[(X['val'][:, feature_index], y['val'])],
   eval_metric=wrmsse_val_eval_metric,
   early_stopping_rounds=50,
   verbose=10,
)

Training until validation scores don't improve for 50 rounds.
[10]	valid_0's WRMSSE: 2.14457
[20]	valid_0's WRMSSE: 1.23637
[30]	valid_0's WRMSSE: 0.92745
[40]	valid_0's WRMSSE: 0.796905
[50]	valid_0's WRMSSE: 0.738645
[60]	valid_0's WRMSSE: 0.705954
[70]	valid_0's WRMSSE: 0.672542
[80]	valid_0's WRMSSE: 0.658003
[90]	valid_0's WRMSSE: 0.651263
[100]	valid_0's WRMSSE: 0.645419
[110]	valid_0's WRMSSE: 0.64389
[120]	valid_0's WRMSSE: 0.641796
[130]	valid_0's WRMSSE: 0.639633
[140]	valid_0's WRMSSE: 0.638194
[150]	valid_0's WRMSSE: 0.634799
[160]	valid_0's WRMSSE: 0.631466
[170]	valid_0's WRMSSE: 0.629713
[180]	valid_0's WRMSSE: 0.628007
[190]	valid_0's WRMSSE: 0.627192
[200]	valid_0's WRMSSE: 0.62594
[210]	valid_0's WRMSSE: 0.62499
[220]	valid_0's WRMSSE: 0.624935
[230]	valid_0's WRMSSE: 0.624303
[240]	valid_0's WRMSSE: 0.624247
[250]	valid_0's WRMSSE: 0.623831
[260]	valid_0's WRMSSE: 0.6238
[270]	valid_0's WRMSSE: 0.623123
[280]	valid_0's WRMSSE: 0.623807
[290]	valid_0's WRMSSE: 0.62305

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              metric='None', min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=500, n_jobs=-1, num_leaves=19,
              objective='mse', random_state=None, reg_alpha=0.0, reg_lambda=0.0,
              silent=True, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0)

In [ ]:
wrmsse_test_eval_metric = WRMSSEEvalMetric(
    test_evaluator,
    ids,
    verbose_step=10
)

In [ ]:
y_pred = regressor.predict(X['test'][:, feature_index])
wrmsse_test_eval_metric(0, y_pred)

('WRMSSE', 0.525254585889248, False)

In [ ]:
rp = RecursivePredictor(header, regressor, label_codes_dict, 1540, feature_index)
y_pred = rp.get_recursive_prediction(X['test'])
wrmsse_test_eval_metric = WRMSSEEvalMetric(
    test_evaluator,
    label_codes_dict,
    verbose_step=10
)
wrmsse_test_eval_metric(0, y_pred)

('WRMSSE', 0.6144906504492261, False)

In [ ]:
X_train = np.load(f'data/X_train_evaluation.npy')

In [ ]:
X_test = np.load(f'data/X_test_evaluation.npy')

In [ ]:
y = X_train[:, 5]

In [ ]:
regressor = LGBMRegressor(
    n_estimators=500,
    learning_rate=0.1,
    num_leaves=19,
    objective='mse',
    metric="None"
)

In [ ]:
regressor.fit(
   X_train[:, feature_index], y
)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              metric='None', min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=500, n_jobs=-1, num_leaves=19,
              objective='mse', random_state=None, reg_alpha=0.0, reg_lambda=0.0,
              silent=True, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0)

In [ ]:
rp = RecursivePredictor(header, regressor, label_codes_dict, 1540, feature_index)

In [ ]:
y_pred = rp.get_recursive_prediction(X_test)

In [ ]:
import pandas as pd

In [ ]:
submit = pd.read_csv('sample_submission.csv')

In [ ]:
submit.iloc[:30490, 1:29] = y_pred.reshape(-1, 28)[ids]

In [ ]:
submit.to_csv('submit.csv')